Test API

In [1]:
import pandas as pd
from pycoingecko import CoinGeckoAPI

# Initialiser le client CoinGeckoAPI
cg = CoinGeckoAPI()

# Paramètres pour la requête API
parameters = {
    'vs_currency': 'usd',
    'order': 'market_cap_desc',
    'per_page': 100,
    'page': 1,
    'sparkline': False,
    'locale': 'en'
}

# Récupérer les données du marché
coin_market_data = cg.get_coins_markets(**parameters)

# Convertir les données en DataFrame pour une meilleure visualisation
df = pd.DataFrame(coin_market_data)

# Sélectionner uniquement la colonne 'name' pour afficher les noms des cryptomonnaies
df_names = df[['name']]

# Afficher les noms des cryptomonnaies
print(df_names)

             name
0         Bitcoin
1        Ethereum
2          Tether
3             BNB
4          Solana
..            ...
95            NEO
96   Pyth Network
97        Conflux
98  WhiteBIT Coin
99         Helium

[100 rows x 1 columns]


In [2]:
from pathlib import Path
from pycoingecko import CoinGeckoAPI
import pandas as pd
import numpy as np
import ta
import matplotlib.pyplot as plt

Pour l'IA

In [3]:
def fetch_data(category_id="artificial-intelligence", days="max", interval="daily", base_dir="../../database/API/Coingecko/AI"):
    cg = CoinGeckoAPI()
    base_dir = Path(base_dir).resolve()
    base_dir.mkdir(parents=True, exist_ok=True)

    # Récupérer les cryptomonnaies de la catégorie spécifiée par l'ID
    coins_in_category = cg.get_coins_markets(vs_currency='usd', category=category_id)

    for coin in coins_in_category:
        coin_id = coin['id']
        print(f"Téléchargement des données pour {coin_id}.")
        market_data = cg.get_coin_market_chart_by_id(id=coin_id, vs_currency='usd', days=days, interval=interval)
        prices = market_data['prices']
        df = pd.DataFrame(prices, columns=['timestamp', coin_id])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df.set_index('timestamp', inplace=True)
        cache_path = base_dir / f"{coin_id}_{days}_{interval}.csv"
        df.to_csv(cache_path)

fetch_data(category_id="artificial-intelligence")

Téléchargement des données pour bittensor.
Téléchargement des données pour render-token.
Téléchargement des données pour fetch-ai.
Téléchargement des données pour singularitynet.
Téléchargement des données pour akash-network.
Téléchargement des données pour corgiai.
Téléchargement des données pour aioz-network.
Téléchargement des données pour ocean-protocol.
Téléchargement des données pour golem.
Téléchargement des données pour arkham.
Téléchargement des données pour origintrail.
Téléchargement des données pour nosana.
Téléchargement des données pour paal-ai.
Téléchargement des données pour 0x0-ai-ai-smart-contract.
Téléchargement des données pour delysium.
Téléchargement des données pour iexec-rlc.
Téléchargement des données pour autonolas.
Téléchargement des données pour rss3.
Téléchargement des données pour oraichain-token.
Téléchargement des données pour numeraire.
Téléchargement des données pour everipedia.
Téléchargement des données pour covalent.
Téléchargement des données pour 

Pour le Top100

In [4]:
def fetch_top_100_data(vs_currency='usd', base_dir='../../database/API/Coingecko/Top100'):
    cg = CoinGeckoAPI()
    base_dir = Path(base_dir).resolve()
    base_dir.mkdir(parents=True, exist_ok=True)
    
    # Paramètres pour la requête API
    parameters = {
        'vs_currency': vs_currency,
        'order': 'market_cap_desc',
        'per_page': 100,
        'page': 1,
        'sparkline': False,
        'price_change_percentage': '1h,24h,7d'
    }

    # Récupérer les données du marché pour le top 100
    coin_market_data = cg.get_coins_markets(**parameters)

    # Boucle pour enregistrer les données de chaque cryptomonnaie
    for coin in coin_market_data:
        coin_id = coin['id']
        print(f"Téléchargement des données pour {coin_id}.")
        market_data = cg.get_coin_market_chart_by_id(id=coin_id, vs_currency=vs_currency, days='max', interval='daily')
        prices = market_data['prices']
        df = pd.DataFrame(prices, columns=['timestamp', coin_id])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df.set_index('timestamp', inplace=True)
        cache_path = base_dir / f"{coin_id}_max_daily.csv"
        df.to_csv(cache_path)

fetch_top_100_data()

Téléchargement des données pour bitcoin.
Téléchargement des données pour ethereum.
Téléchargement des données pour tether.
Téléchargement des données pour binancecoin.
Téléchargement des données pour solana.
Téléchargement des données pour ripple.
Téléchargement des données pour staked-ether.
Téléchargement des données pour usd-coin.
Téléchargement des données pour cardano.
Téléchargement des données pour dogecoin.
Téléchargement des données pour shiba-inu.
Téléchargement des données pour avalanche-2.
Téléchargement des données pour polkadot.
Téléchargement des données pour chainlink.
Téléchargement des données pour tron.
Téléchargement des données pour matic-network.
Téléchargement des données pour wrapped-bitcoin.
Téléchargement des données pour the-open-network.
Téléchargement des données pour uniswap.
Téléchargement des données pour bitcoin-cash.
Téléchargement des données pour litecoin.
Téléchargement des données pour near.
Téléchargement des données pour internet-computer.
Téléch

In [5]:
class MADStrategy:
    def __init__(self, category="Top100"):
        base_path = "../../database/API/Coingecko"
        self.base_dir = Path(base_path, category).resolve()
        self.dataframes = {}
        self.wallet_history = []

    def load_data(self, coin_ids, days="max", interval="daily"):
        for coin_id in coin_ids:
            cache_path = self.base_dir / f"{coin_id}_{days}_{interval}.csv"
            if cache_path.exists():
                df = pd.read_csv(cache_path, index_col='timestamp', parse_dates=True)
                self.dataframes[coin_id] = df

    def calculate_ema_and_mad(self, periods=[21, 200]):
        mad_scores = {}
        for coin, df in self.dataframes.items():
            for period in periods:
                ema_key = f"EMA_{period}"
                df[ema_key] = ta.trend.ema_indicator(close=df[coin], window=period)
            df['MAD'] = df[f'EMA_{periods[0]}'] / df[f'EMA_{periods[1]}']
            self.dataframes[coin] = df
            # Enregistrer la dernière valeur de MAD, si elle n'est pas nan
            if not pd.isna(df['MAD'].iloc[-1]):
                mad_scores[coin] = df['MAD'].iloc[-1]

        # Trier les scores MAD et sélectionner le top 10, en excluant les valeurs nan
        top_mad_coins = sorted(mad_scores.items(), key=lambda x: x[1], reverse=True)[:15]
        return top_mad_coins


Choisir "Top100" ou "AI" 

In [6]:
from pathlib import Path

# Choisissez la catégorie ici: "Top100" ou "AI"
selected_category = "Top100"

strategy = MADStrategy(category=selected_category)

data_dir = strategy.base_dir
coin_ids = [f.stem.split('_')[0] for f in data_dir.glob("*.csv")]
strategy.load_data(coin_ids)

top_mad_coins = strategy.calculate_ema_and_mad()

# Affichage des résultats
print(f"Top 15 Cryptocurrencies by MAD Ratio in {selected_category}:")
for coin, mad_ratio in top_mad_coins:
    print(f"{coin}: MAD Ratio = {mad_ratio}")

Top 15 Cryptocurrencies by MAD Ratio in Top100:
floki: MAD Ratio = 3.0405805367382883
pepe: MAD Ratio = 2.9695074754632906
fetch-ai: MAD Ratio = 2.611374321797042
arweave: MAD Ratio = 2.485488990208029
ribbon-finance: MAD Ratio = 2.341434158978568
singularitynet: MAD Ratio = 2.327203633033273
bonk: MAD Ratio = 2.318243971071554
worldcoin-wld: MAD Ratio = 2.1614199124521005
shiba-inu: MAD Ratio = 2.06272527584079
bittensor: MAD Ratio = 2.00853511540358
akash-network: MAD Ratio = 1.989827731279038
theta-token: MAD Ratio = 1.978629901594772
render-token: MAD Ratio = 1.965938143899637
axelar: MAD Ratio = 1.9301702432290977
blockstack: MAD Ratio = 1.8848426915393974
